In [1]:
import pandas as pd


In [2]:
df_distances = pd.read_csv("../../results_demo/T50/FFT/Group_0/distance_estimator.csv")
df_distances

,id_seq1,target_seq1,id_seq2,target_seq2,distance
0,sequence_53082,55.0,sequence_19402,43.0,38663.916973
1,sequence_53082,55.0,sequence_20054,49.0,38403.350335
2,sequence_53082,55.0,sequence_20048,39.8,37989.963358
3,sequence_53082,55.0,sequence_20036,52.9,37428.543420
4,sequence_53082,55.0,sequence_19525,48.8,37811.339553
...,...,...,...,...,...
67855,sequence_19401,47.4,sequence_53056,52.0,19825.366654
67856,sequence_19401,47.4,sequence_19347,58.9,16852.986398
67857,sequence_19401,47.4,sequence_19379,50.3,10269.414721
67858,sequence_19401,47.4,sequence_19348,46.7,27063.790047


In [17]:
#filtramos las distancias para trabajar con solo una parte de ellas -> nos interesan solo las más cercanas
import numpy as np

q1 = np.quantile(df_distances['distance'], .25)
q3 = np.quantile(df_distances['distance'], .75)

print(q3, q1)
IQR = q3 - q1
IQR = 1 * IQR

low_freq = q1 - IQR
high_freq = q3 + IQR

print(low_freq, high_freq)

df_filter = df_distances.loc[df_distances['distance'] <= q3]
df_filter = df_filter.reset_index()

34543.31569245595 26205.313375488146
17867.31105852034 42881.31800942376


In [18]:
df_filter

,index,id_seq1,target_seq1,id_seq2,target_seq2,distance
0,11,sequence_53082,55.0,sequence_53075,51.6,34234.861491
1,16,sequence_53082,55.0,sequence_53062,49.1,27381.373431
2,27,sequence_53082,55.0,sequence_53085,51.2,32470.472196
3,32,sequence_53082,55.0,sequence_19369,47.9,32665.321140
4,38,sequence_53082,55.0,sequence_20064,47.3,31204.318748
...,...,...,...,...,...,...
50891,67855,sequence_19401,47.4,sequence_53056,52.0,19825.366654
50892,67856,sequence_19401,47.4,sequence_19347,58.9,16852.986398
50893,67857,sequence_19401,47.4,sequence_19379,50.3,10269.414721
50894,67858,sequence_19401,47.4,sequence_19348,46.7,27063.790047


In [5]:
pip install python-louvain
pip install networkx

  Using cached python_louvain-0.16-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [3]:
import community as community_louvain
import networkx as nx

In [19]:
graph_data = nx.Graph()

id_list1 = df_filter['id_seq1'].to_list()
id_list2 = df_filter['id_seq2'].to_list()

nodos = id_list1 + id_list2
nodos = list(set(nodos))

#cargar nodos
for node in nodos:
    graph_data.add_node(node)

#cargar aristas
for i in range(len(df_filter)):
    graph_data.add_edge(df_filter['id_seq1'][i],
        df_filter['id_seq2'][i],
        weigth=df_filter['distance'][i]
    )

In [23]:
partition = community_louvain.best_partition(graph_data)
modularity_value = round(community_louvain.modularity(partition, graph_data), 3)

print("Modularity: ", modularity_value)

Modularity:  0.065


In [26]:
#obtenemos los grupos
matrix_group = []
for element in partition:
    row = [element, partition[element]]
    matrix_group.append(row)
results = pd.DataFrame(matrix_group, columns=['id', 'label'])

results.to_csv("../../results_demo/T50/FFT/Group_0/graph_clustering_sequences.csv", index=False)